In [1]:
import numpy as np
import cv2

import PIL.Image as image
import os

import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.models import Sequential, Model

from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

### mobilenet is a pretrained model that is trained on 1.4 million images and 1000 different classes

- images are in 224x224 dimension

In [2]:
IMAGE_SHAPE = (224,224)
model_224 = MobileNetV2(
    include_top=True,
    input_shape=IMAGE_SHAPE+(3,),
    weights='imagenet'
)
# model_224.trainable = False

In [3]:
image_lables = []
with open("ImageNetLabels.txt", "r") as f:
    image_labels = f.read().splitlines()
image_labels[:5]

['background', 'tench', 'goldfish', 'great white shark', 'tiger shark']

In [4]:
# dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
# data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url, cache_dir='.', untar=True)

In [5]:
import pathlib
data_dir = pathlib.Path('datasets/flower_photos')
data_dir

PosixPath('datasets/flower_photos')

In [6]:
flowers_images_dict = {
    'roses' : list(data_dir.glob("roses/*")),
    'tulips' : list(data_dir.glob("tulips/*")),
    'daisy' : list(data_dir.glob("daisy/*")),
    'dandelion' : list(data_dir.glob("dandelion/*")),
    'sunflowers' : list(data_dir.glob("sunflowers/*")),
}

In [7]:
flowers_label_dict = {
    'roses' : 0,
    'tulips' : 1,
    'daisy' : 2,
    'dandelion' : 3,
    'sunflowers' : 4,
}

In [8]:
str(flowers_images_dict['roses'][0])

'datasets/flower_photos/roses/16209331331_343c899d38.jpg'

In [9]:
img = cv2.imread(str(flowers_images_dict['roses'][0]))
img.shape

(243, 500, 3)

### before training model we need to make sure all images are of same size
- here we should reshape image to 224x224

In [10]:
X, y = [], []

for flower_name, images in flowers_images_dict.items() :
    for image in images:
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img, IMAGE_SHAPE)
        X.append(resized_img)
        y.append(flowers_label_dict[flower_name])

### we should normalize the image by dividing it by 255 to bring the value to 0 to 1

In [11]:
X = np.array(X)
y = np.array(y)

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [13]:
X_train_scaled = X_train / 255
X_test_scaled = X_test / 255

In [14]:
predicted = model_224.predict(np.array([X[0], X[1], X[2]]))
predicted = np.argmax(predicted, axis=1)
predicted

array([794, 794, 721])

In [15]:
y[3]

0

In [16]:
image_labels[791]

'shopping basket'

## now lets import mobilenet model without the last layer 

In [126]:
model_224_without_top_layer = MobileNetV2(
    include_top=False,
    input_shape=IMAGE_SHAPE+(3,)
)
model_224_without_top_layer.trainable = False

In [127]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(model_224_without_top_layer.layers))
num_of_layers_to_train = 0
# Fine-tune from this layer onwards
fine_tune_from_layer_number = len(model_224_without_top_layer.layers) - 1 - num_of_layers_to_train

# Freeze all the layers before the `fine_tune_at` layer
for layer in model_224_without_top_layer.layers[:fine_tune_from_layer_number]:
  layer.trainable = False

Number of layers in the base model:  154


In [128]:
model_224_without_top_layer.compile(
    optimizer = tf.keras.optimizers.Adam(),
    loss = tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics = ['acc']
)

# model_224_without_top_layer.compile('adam',
#               loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
#               metrics=['accuracy'])

In [129]:
model_224_without_top_layer.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_7[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                              

In [130]:
#updating weights

# layer names
pre_model_config = model_224_without_top_layer.get_config()
layer_names = [pre_model_config['layers'][x]['name'] for x in range(len(pre_model_config['layers']))]
layer_names



['input_7',
 'Conv1',
 'bn_Conv1',
 'Conv1_relu',
 'expanded_conv_depthwise',
 'expanded_conv_depthwise_BN',
 'expanded_conv_depthwise_relu',
 'expanded_conv_project',
 'expanded_conv_project_BN',
 'block_1_expand',
 'block_1_expand_BN',
 'block_1_expand_relu',
 'block_1_pad',
 'block_1_depthwise',
 'block_1_depthwise_BN',
 'block_1_depthwise_relu',
 'block_1_project',
 'block_1_project_BN',
 'block_2_expand',
 'block_2_expand_BN',
 'block_2_expand_relu',
 'block_2_depthwise',
 'block_2_depthwise_BN',
 'block_2_depthwise_relu',
 'block_2_project',
 'block_2_project_BN',
 'block_2_add',
 'block_3_expand',
 'block_3_expand_BN',
 'block_3_expand_relu',
 'block_3_pad',
 'block_3_depthwise',
 'block_3_depthwise_BN',
 'block_3_depthwise_relu',
 'block_3_project',
 'block_3_project_BN',
 'block_4_expand',
 'block_4_expand_BN',
 'block_4_expand_relu',
 'block_4_depthwise',
 'block_4_depthwise_BN',
 'block_4_depthwise_relu',
 'block_4_project',
 'block_4_project_BN',
 'block_4_add',
 'block_5_e

In [131]:
len(model_224_without_top_layer.layers)
model_224_without_top_layer.layers[153]
layer_names[153]

'out_relu'

In [132]:
for layer in model_224.layers:
    if layer.name in layer_names:
        if layer.get_weights() != [] :
            target_layer = model_224_without_top_layer.get_layer(layer.name)
            
            weight = layer.get_weights()
            # bias = layer.get_weights()[1]

            target_layer.set_weights(weight)
            target_layer.trainable = False
        

In [160]:
num_of_flowers = 5
model = tf.keras.Sequential([
    model_224_without_top_layer,
    tf.keras.layers.GlobalAveragePooling2D(),
    # tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(num_of_flowers)
])

model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d_7   (None, 1280)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_6 (Dense)             (None, 5)                 6405      
                                                                 
Total params: 2,264,389
Trainable params: 6,405
Non-trainable params: 2,257,984
_________________________________________________________________


In [161]:
model.layers[0].layers[152].get_weights()

[array([1.2102375, 1.1382099, 1.1937536, ..., 1.1645414, 1.1686966,
        1.2112191], dtype=float32),
 array([-1.1921003, -1.0592812, -1.2243685, ..., -1.1624321, -1.1511711,
        -1.1495001], dtype=float32),
 array([ 3.9949318e-05, -2.6198127e-04, -5.3372354e-05, ...,
        -4.4540127e-05, -9.6894546e-05,  5.3597880e-05], dtype=float32),
 array([0.28292483, 0.2462233 , 0.2856205 , ..., 0.274729  , 0.28398702,
        0.2686955 ], dtype=float32)]

In [162]:
model.layers[0].layers[152].get_weights()[0]/4

array([0.30255938, 0.28455248, 0.2984384 , ..., 0.29113534, 0.29217416,
       0.30280477], dtype=float32)

In [163]:
# i = 0
update_weight_layer = model.layers[0].layers[152]
old_weights = update_weight_layer.get_weights()

updated_weights = [] 
for old_weight in old_weights:
    average = 0
    for weight in old_weight:
        sum = 0 + weight
    average = sum/len(old_weight)
    for x in range(len(old_weight)):
        old_weight[x] = average
    updated_weights.append(old_weight)
updated_weights
# update_weight_layer.set_weights(updated_weights)

[array([0.00094626, 0.00094626, 0.00094626, ..., 0.00094626, 0.00094626,
        0.00094626], dtype=float32),
 array([-0.00089805, -0.00089805, -0.00089805, ..., -0.00089805,
        -0.00089805, -0.00089805], dtype=float32),
 array([4.1873342e-08, 4.1873342e-08, 4.1873342e-08, ..., 4.1873342e-08,
        4.1873342e-08, 4.1873342e-08], dtype=float32),
 array([0.00020992, 0.00020992, 0.00020992, ..., 0.00020992, 0.00020992,
        0.00020992], dtype=float32)]

In [164]:
model.layers[0].layers[152].set_weights(updated_weights)

In [165]:
model.layers[0].layers[152].get_weights()

[array([0.00094626, 0.00094626, 0.00094626, ..., 0.00094626, 0.00094626,
        0.00094626], dtype=float32),
 array([-0.00089805, -0.00089805, -0.00089805, ..., -0.00089805,
        -0.00089805, -0.00089805], dtype=float32),
 array([4.1873342e-08, 4.1873342e-08, 4.1873342e-08, ..., 4.1873342e-08,
        4.1873342e-08, 4.1873342e-08], dtype=float32),
 array([0.00020992, 0.00020992, 0.00020992, ..., 0.00020992, 0.00020992,
        0.00020992], dtype=float32)]

In [166]:
model.compile(
    optimizer = 'adam',
    loss = tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics = ['acc']
)

In [167]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d_7   (None, 1280)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_6 (Dense)             (None, 5)                 6405      
                                                                 
Total params: 2,264,389
Trainable params: 6,405
Non-trainable params: 2,257,984
_________________________________________________________________


In [168]:
model.optimizer.learning_rate

<tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.001>

In [169]:
model.fit(X_train_scaled, y_train, epochs=15)

Epoch 1/15
86/86 [==============================] - 65s 717ms/step - loss: 3.9481 - acc: 0.3118
Epoch 2/15
86/86 [==============================] - 62s 716ms/step - loss: 1.6089 - acc: 0.2013
Epoch 3/15
86/86 [==============================] - 62s 715ms/step - loss: 1.6089 - acc: 0.2002
Epoch 4/15
86/86 [==============================] - 61s 712ms/step - loss: 1.6089 - acc: 0.2002
Epoch 5/15
86/86 [==============================] - 61s 704ms/step - loss: 1.6089 - acc: 0.2002
Epoch 6/15
86/86 [==============================] - 61s 705ms/step - loss: 1.6089 - acc: 0.2002
Epoch 7/15
86/86 [==============================] - 61s 706ms/step - loss: 1.6089 - acc: 0.2002
Epoch 8/15
86/86 [==============================] - 62s 717ms/step - loss: 1.6089 - acc: 0.2002
Epoch 9/15
86/86 [==============================] - 61s 708ms/step - loss: 1.6089 - acc: 0.2002
Epoch 10/15
86/86 [==============================] - 61s 710ms/step - loss: 1.6089 - acc: 0.2002
Epoch 11/15
86/86 [====================

In [170]:
model.evaluate(X_test_scaled, y_test)

29/29 [==============================] - 22s 671ms/step - loss: 1.6094 - acc: 0.1863


[1.609437346458435, 0.18627451360225677]

In [ ]:
predicted = model.predict(X_test)
predicted = np.argmax(predicted, axis=1)
predicted

array([4, 3, 3, 0, 3, 3, 0, 0, 3, 3, 0, 4, 3, 3, 0, 3, 0, 0, 1, 1, 0, 3,
       4, 3, 3, 3, 1, 3, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 4,
       3, 0, 3, 0, 0, 0, 3, 3, 3, 0, 3, 4, 1, 0, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 1, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 1, 0, 3, 3,
       3, 0, 3, 1, 0, 3, 0, 0, 3, 3, 3, 3, 3, 0, 0, 3, 3, 0, 3, 3, 3, 3,
       3, 0, 3, 3, 4, 4, 3, 0, 4, 0, 3, 3, 4, 1, 1, 3, 0, 3, 3, 4, 3, 3,
       3, 3, 3, 3, 3, 0, 3, 3, 3, 4, 3, 3, 4, 3, 3, 3, 3, 3, 3, 3, 0, 3,
       3, 3, 3, 3, 1, 4, 3, 3, 4, 3, 3, 3, 0, 1, 3, 4, 1, 3, 4, 3, 3, 3,
       3, 4, 3, 3, 0, 0, 3, 3, 3, 3, 3, 0, 3, 0, 3, 3, 3, 3, 3, 1, 0, 4,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 3, 3, 3, 3, 0,
       4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 0, 3, 3, 4, 4, 3, 0, 3, 3, 3,
       4, 4, 0, 4, 3, 1, 3, 3, 4, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0,
       3, 4, 0, 3, 0, 0, 4, 3, 0, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 4, 3,
       4, 0, 3, 0, 1, 3, 0, 0, 0, 3, 3, 0, 3, 1, 0,

In [ ]:
predicted[0]

4

In [ ]:
flowers_label_dict

{'roses': 0, 'tulips': 1, 'daisy': 2, 'dandelion': 3, 'sunflowers': 4}

In [ ]:
y_test[0]

1

In [ ]:
predicted[13]

3

In [ ]:
y_test[13]

3

In [ ]:
model.layers